In [1]:
# In[1]:
import time
t1 = time.time()

#importando as bibliotecas 
import torch
import torch_geometric
import numpy as np
import sys 
from torch_geometric.data import Data
import torch.nn.functional as F 
from torch_geometric.nn import GCNConv 
import statistics as sts
import matplotlib.pyplot as plt

sys.path.insert(1, '/home/marina/InCi/')
from cbirdb import acbirdb,cbirdbDistMatrix,evaluator,unsuperalg,recknngraph

execucoes = 10

def DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs): 

    # máscaras 

    train_mask = [] #máscaras de treino - true = treino 
    val_mask = [] #valores das máscaras 
    test_mask = [] #máscaras de testes 
    y = [] # y = vetor de "gabarito" 

    for i in range (pN):# para cada imagem i: 
        y.append(i // classSize) # definindo o vetor de classes 
        d = i % classSize # d = resto do número da imagem dividido pela classe 
        if (d<trPerClass): # se d é menor que o número de imagens de treino por classe, então: 
            valueTr = True # treino = true 
            valueVal = False # teste = false 
        else: # d é maior ou igual ao número de imagens de treino por classe: 
            valueTr = False # treino = false 
            if (d<(valPerClass+trPerClass)): 
                valueVal = True # teste = true 
            else:
                valueVal = False #teste = false 
        valueTest = (not valueTr) and (not valueVal) 

        train_mask.append (valueTr) # criando os vetores de máscaras 
        val_mask.append(valueVal) #vetor de validação 
        test_mask.append(valueTest)

    # A torch.Tensor is a multi-dimensional matrix containing elements of a single data type.
    train_mask = torch.tensor (train_mask) #torch 
    val_mask = torch.tensor (val_mask)
    test_mask = torch.tensor (test_mask)
    y = torch.tensor (y)

    
    #-------------------------------- Feature Matrix ------------------------------------- 
    #lendo a matriz de features 

    #print ('Reading feature matrix ...')
    feat_matrix_file = "feat-matrix.txt"
    x = np.loadtxt(feat_matrix_file, delimiter=",")
    #print(x.shape)
    x = torch.tensor(x)
    #print (x)
    
    # Indexação das Bordas 

    #arquivo lista 
    pListFile = "list.txt"
    #arquivo dist matrix
    pDSimFile = DistM #"/home/marina/venv/oxford17flowers-caffe-fc7_matrix.txt"


    #print ('Reading DB ...')
    db = cbirdbDistMatrix(pListFile,pDSimFile,pN)

    #print ('Making edge list ...')
    edge_index = []

    for qimg in db.getList(): #para cada imagem da lista 
        #print (qimg)
        crl = db.getRL(qimg) # mais próximos no ranked list 
        qindex = db.getIndexOf(qimg) 
        for i in range(pK): # cria arestas entre as k posições mais próximas 
            cindex = db.getIndexOf (crl[i])
            edge_index.append([qindex,cindex])

    #print(edge_index)

    #passa para o torch 
    # A torch.Tensor is a multi-dimensional matrix containing elements of a single data type.
    edge_index = torch.tensor(edge_index)
    edge_index = edge_index.t().contiguous()

    #print (edge_index)
    
    #--------------------------------- Data Object  -----------------------------------------
    #data são as informações do grafo completo (eu acho) *_* 

    #print ('Loading data object...')
    data = Data(x=x.float(), edge_index=edge_index, y=y, test_mask=test_mask, train_mask=train_mask,  val_mask=val_mask  )


    #--------------------------------- Graph Neural Network Defiition  -----------------------------------------

    # rede neural: NÃO MEXA!!!!!!!!!!

    #print ('Defining GCN model...')

    class Net(torch.nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = GCNConv(pNFeatures, pNNeurons) #dataset.num_node_features
            self.conv2 = GCNConv(pNNeurons, numberOfClasses) #dataset.num_classes

        def forward(self, data):
            x, edge_index = data.x, data.edge_index

            x = self.conv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training)
            x = self.conv2(x, edge_index)

            return F.log_softmax(x, dim=1)



    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #device = torch.device('cpu')
    model = Net().to(device)
    #data = dataset[0].to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=pLR, weight_decay=5e-4)

    #print ('Training...')

    model.train() # treinando com a rede neural 
    for epoch in range(pNEpochs):	
        #print("Training epoch: ", epoch)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()


    # aqui ele avalia o modelo, com base nas máscaras de teste e treino que definimos no começo     
    model.eval()
    _, pred = model(data).max(dim=1)
    correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
    acc = correct / pN #data.test_mask.sum().item() # esse acc é o mesmo da soccer? é um descritor? 

    #print('Accuracy: {:.4f}'.format(acc)) 
    return(acc)


# In[ ]:


# informações do dataset
pN = 1360 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 15 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 35 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.001 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 32 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 200 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos 
DistM = "oxford17flowers-caffe-fc7_matrix.txt"

'''
vpk = []
vpn = []

dpk = []
dpn = []

for i in range(execucoes): 
    vpk.append(0)
    vpn.append(0)

print(vpk)
print(vpn)

for i in range(execucoes):
    print("Execução num ", i)
    pK = 25
    pNNeurons = 24
    VariaKs = []
    VariaNeuronios = []

    while pK <= 50: 
        print("K: ", pK)
        m = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
        VariaKs.append(m)
        pK = pK + 5        

    while pNNeurons <= 36: 
        print("Neurônio: ", pNNeurons)
        c = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
        VariaNeuronios.append(c)
        pNNeurons = pNNeurons + 4
        
    dpk.append(VariaKs)
    dpn.append(VariaNeuronios)

    print("O PROBLEMA: ")
    print(len(VariaKs))
    print(len(VariaNeuronios))
    print(vpk)
    print(vpn)
    
    for k in range(len(VariaKs)):
        vpk[k] = vpk[k] + VariaKs[k]
        
    for k in range(len(VariaNeuronios)):
        vpn[k] = vpn[k] + VariaNeuronios[k]
        
print("\nFim do Loop")

for k in range(len(vpk)):
    vpk[k] = vpk[k]/execucoes
    
for k in range(len(vpn)):
    vpn[k] = vpn[k]/execucoes

print("\ndpk: ", dpk)
print("\ndpn: ", dpn)


# In[ ]:


'''
'''
t = np.asarray([[1, 2, 3], ['a', 'b', 'c']])
print(t.shape)
print(t[:, 0])
'''
'''
media_k = []
media_N = []
j = 0 
stdevK = []
stdevN = []

dpk = np.asarray(dpk)
print(dpk.shape)
for i in range( dpk.shape[1] ):# para cada linha (10)
    for coluna in dpk[:, i]: #para elemento da coluna
        j = j + coluna # j é a soma de todos os elementos da coluna 
    # saiu do for 
    stdevK.append(sts.stdev(dpk[:, i])) # desvio padrão da coluna 
    j = j/dpk.shape[1] # j é a média da coluna 
    media_k.append(j)
    j = 0
    
j = 0
dpn = np.asarray(dpn)
print(dpn.shape)
for i in range( dpn.shape[1] ):# para cada linha (10)
    for coluna in dpn[:, i]: #para elemento da coluna
        j = j + coluna # j é a soma de todos os elementos da coluna 
    # saiu do for 
    stdevN.append(sts.stdev(dpn[:, i])) # desvio padrão da coluna 
    j = j/dpn.shape[1] # j é a média da coluna 
    media_N.append(j)
teste = [1, 2, 3, 4, 5, 6]
media_k = np.asarray(media_k)
media_N = np.asarray(media_N)
print(media_k)
print(media_N)
stdevK = np.asarray(stdevK)
stdevN = np.asarray(stdevN)
print(stdevK)
print(stdevN)


# In[ ]:


import matplotlib
import matplotlib.pyplot as plt

plt.show( plt.errorbar(x = [25, 30, 35, 40, 45, 50], y = media_k, yerr = stdevK, fmt = '-o') )
plt.show(plt.errorbar(x = [24, 28, 32, 36], y = media_N, yerr = stdevN, fmt = '-o'))

'''
# In[ ]:


'''gerar uma nova matriz de distância, com base nas features '''
'''variar o k de 25 a 50, de 5 em 5 
variar neurônios de 24 a 36, de 4 em 4 
'''


# In[ ]:


# informações do dataset
pN = 1360 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 15 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 40 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.001 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 36 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 200 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos 


DistM = "nova_dist_matrix.txt"
Media_NM = 0
dp_original = []


for i in range(10): 
    NM = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_original.append(NM)
    Media_NM = Media_NM + NM


# In[ ]:


DistM = "Nova_Matrix_UDLF.txt"
PoD = 0 
dp_udlf = []
for i in range(11): 
    print(i)
    AP = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_udlf.append(AP)
    PoD = PoD + AP


# In[ ]:


DistM = "oxford17flowers-caffe-fc7_matrix.txt"
PoDo = 0 
dp_udlf = []
for i in range(10): 
    print(i)
    APO = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_udlf.append(APO)
    PoDo = PoDo + APO


# In[ ]:


Media_NM = Media_NM/execucoes 
print("Nova Matriz sem UDLF: ", Media_NM, "\nDesvio Padrão: ", sts.stdev(dp_original))
PoD = PoD/execucoes
print("Nova Matriz com UDLF: ", PoD, "\nDesvio Padrão: ", sts.stdev(dp_udlf))
print("Antiga Matriz (café) sem UDLF: ", PoDo/execucoes)


# In[ ]:
t2 = time.time()
print("Tempo de execução: ", (t2-t1)/60 )

0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
Nova Matriz sem UDLF:  0.05830882352941176 
Desvio Padrão:  0.016165511736532193
Nova Matriz com UDLF:  0.32125 
Desvio Padrão:  0.15383419838280424
Antiga Matriz (café) sem UDLF:  0.17911764705882352
Tempo de execução:  20.856146800518037


In [2]:

# informações do dataset
pN = 1360 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 15 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 80 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.001 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 36 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 200 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos 


DistM = "/home/marina/Documents/H/oxford17flowers-caffe-fc7_matrix.txt"
Media_NM = 0
dp_original = []

for i in range(10): 
    print(i)
    NM = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_original.append(NM)
    Media_NM = Media_NM + NM

Media_NM = Media_NM/execucoes 
print("Café sem UDLF: ", Media_NM, "\nDesvio Padrão: ", sts.stdev(dp_original))

0
1
2
3
4
5
6
7
8
9
Café sem UDLF:  0.14985294117647058 
Desvio Padrão:  0.16067173844694418


In [3]:

# informações do dataset
pN = 1360 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 15 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 8 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.001 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 36 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 200 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos 


DistM = "/home/marina/Documents/H/oxford17flowers-caffe-fc7_matrix.txt"
Media_NM = 0
dp_original = []

for i in range(10): 
    print(i)
    NM = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_original.append(NM)
    Media_NM = Media_NM + NM

Media_NM = Media_NM/execucoes 
print("Café sem UDLF: ", Media_NM, "\nDesvio Padrão: ", sts.stdev(dp_original))

0
1
2
3
4
5
6
7
8
9
Café sem UDLF:  0.17786764705882352 
Desvio Padrão:  0.09336495000206924


In [4]:

# informações do dataset
pN = 1360 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 15 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 35 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.001 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 50 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 200 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos 


DistM = "/home/marina/Documents/H/oxford17flowers-caffe-fc7_matrix.txt"
Media_NM = 0
dp_original = []

for i in range(10): 
    print(i)
    NM = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_original.append(NM)
    Media_NM = Media_NM + NM

Media_NM = Media_NM/execucoes 
print("Café sem UDLF: ", Media_NM, "\nDesvio Padrão: ", sts.stdev(dp_original))

0
1
2
3
4
5
6
7
8
9
Café sem UDLF:  0.3386029411764706 
Desvio Padrão:  0.15561940224920986


In [5]:

# informações do dataset
pN = 1360 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 15 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 35 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.001 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 60 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 200 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos 


DistM = "/home/marina/Documents/H/oxford17flowers-caffe-fc7_matrix.txt"
Media_NM = 0
dp_original = []

for i in range(10): 
    print(i)
    NM = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_original.append(NM)
    Media_NM = Media_NM + NM

Media_NM = Media_NM/execucoes 
print("Café sem UDLF: ", Media_NM, "\nDesvio Padrão: ", sts.stdev(dp_original))

0
1
2
3
4
5
6
7
8
9
Café sem UDLF:  0.36389705882352935 
Desvio Padrão:  0.1576567107875936


In [6]:

# informações do dataset
pN = 1360 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 20 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 35 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.001 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 55 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 200 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos 


DistM = "/home/marina/Documents/H/oxford17flowers-caffe-fc7_matrix.txt"
Media_NM = 0
dp_original = []

for i in range(10): 
    print(i)
    NM = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_original.append(NM)
    Media_NM = Media_NM + NM

Media_NM = Media_NM/execucoes 
print("Café sem UDLF: ", Media_NM, "\nDesvio Padrão: ", sts.stdev(dp_original))

0
1
2
3
4
5
6
7
8
9
Café sem UDLF:  0.3770588235294117 
Desvio Padrão:  0.11035934890185921


In [7]:

# informações do dataset
pN = 1360 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 20 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 35 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.001 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 55 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 200 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos 


DistM = "/home/marina/Documents/H/oxford17flowers-caffe-fc7_matrix.txt"
Media_NM = 0
dp_original = []

for i in range(10): 
    print(i)
    NM = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_original.append(NM)
    Media_NM = Media_NM + NM

Media_NM = Media_NM/execucoes 
print("Café sem UDLF: ", Media_NM, "\nDesvio Padrão: ", sts.stdev(dp_original))

0
1
2
3
4
5
6
7
8
9
Café sem UDLF:  0.3741911764705882 
Desvio Padrão:  0.13656842070983605


In [8]:

# informações do dataset
pN = 1360 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 20 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 35 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.001 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 505 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 200 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos 


DistM = "/home/marina/Documents/H/oxford17flowers-caffe-fc7_matrix.txt"
Media_NM = 0
dp_original = []

for i in range(10): 
    print(i)
    NM = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_original.append(NM)
    Media_NM = Media_NM + NM

Media_NM = Media_NM/execucoes 
print("Café sem UDLF: ", Media_NM, "\nDesvio Padrão: ", sts.stdev(dp_original))

0
1
2
3
4
5
6
7
8
9
Café sem UDLF:  0.5516911764705882 
Desvio Padrão:  0.006950161045766613


In [9]:

# informações do dataset
pN = 1360 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 20 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 35 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.001 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 5000 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 200 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos 


DistM = "/home/marina/Documents/H/oxford17flowers-caffe-fc7_matrix.txt"
Media_NM = 0
dp_original = []

for i in range(10): 
    print(i)
    NM = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_original.append(NM)
    Media_NM = Media_NM + NM

Media_NM = Media_NM/execucoes 
print("Café sem UDLF: ", Media_NM, "\nDesvio Padrão: ", sts.stdev(dp_original))

0
1
2
3
4
5
6


KeyboardInterrupt: 

In [ ]:

# informações do dataset
pN = 1360 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 20 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 35 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.01 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 500 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 200 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos 


DistM = "/home/marina/Documents/H/oxford17flowers-caffe-fc7_matrix.txt"
Media_NM = 0
dp_original = []

for i in range(10): 
    print(i)
    NM = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_original.append(NM)
    Media_NM = Media_NM + NM

Media_NM = Media_NM/execucoes 
print("Café sem UDLF: ", Media_NM, "\nDesvio Padrão: ", sts.stdev(dp_original))

In [ ]:

# informações do dataset
pN = 1360 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 20 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 35 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.0001 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 500 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 200 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos 


DistM = "/home/marina/Documents/H/oxford17flowers-caffe-fc7_matrix.txt"
Media_NM = 0
dp_original = []

for i in range(10): 
    print(i)
    NM = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_original.append(NM)
    Media_NM = Media_NM + NM

Media_NM = Media_NM/execucoes 
print("Café sem UDLF: ", Media_NM, "\nDesvio Padrão: ", sts.stdev(dp_original))

In [ ]:

# informações do dataset
pN = 1360 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 20 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 35 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.005 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 500 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 200 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos 


DistM = "/home/marina/Documents/H/oxford17flowers-caffe-fc7_matrix.txt"
Media_NM = 0
dp_original = []

for i in range(10): 
    print(i)
    NM = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_original.append(NM)
    Media_NM = Media_NM + NM

Media_NM = Media_NM/execucoes 
print("Café sem UDLF: ", Media_NM, "\nDesvio Padrão: ", sts.stdev(dp_original))

In [ ]:

# informações do dataset
pN = 1360 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 20 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 35 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.001 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 500 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 100 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos 


DistM = "/home/marina/Documents/H/oxford17flowers-caffe-fc7_matrix.txt"
Media_NM = 0
dp_original = []

for i in range(10): 
    print(i)
    NM = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_original.append(NM)
    Media_NM = Media_NM + NM

Media_NM = Media_NM/execucoes 
print("Café sem UDLF: ", Media_NM, "\nDesvio Padrão: ", sts.stdev(dp_original))

In [ ]:

# informações do dataset
pN = 1360 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 20 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 35 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.001 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 500 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 300 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos 


DistM = "/home/marina/Documents/H/oxford17flowers-caffe-fc7_matrix.txt"
Media_NM = 0
dp_original = []

for i in range(10): 
    print(i)
    NM = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_original.append(NM)
    Media_NM = Media_NM + NM

Media_NM = Media_NM/execucoes 
print("Café sem UDLF: ", Media_NM, "\nDesvio Padrão: ", sts.stdev(dp_original))